In [ ]:
# Code for fine-tune squeezenet1_1
# YOUR CODE HERE
from torchvision.datasets import ImageFolder
from torch.autograd import Variable
import numpy as np
import scipy.io
import torch
from torch import nn, optim
import torchvision
import torch.nn.functional as F  # a lower level (compared to torch.nn) interface
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import EMNIST
import torchvision.transforms as transforms
import numpy as np
import scipy.io
import torch
import torchvision.datasets
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt

batch_size1 = 20
num_epochs = 20
def calculate_accuracy(model, loader):
    model.eval()
    num_correct, num_samples = 0, 0
    for x, y in loader:
        x_var = Variable(x, volatile=True) # .type(dtype)
        scores = model(x_var)
        _, preds = scores.data.cpu().max(1)
        num_correct += (preds == y).sum()
        num_samples += x.size(0)
        acc = float(num_correct) / num_samples
    return acc

def run_model(model, loss_fn, loader, optimizer):
    model.train()
    for x, y in loader:
        x_var = Variable(x.type(torch.FloatTensor))
        y_var = Variable(y.type(torch.FloatTensor).long())
        scores = model(x_var)
        loss = loss_fn(scores, y_var)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    return loss
    

mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
train_transform = transforms.Compose([transforms.Scale(256), transforms.RandomSizedCrop(224), transforms.RandomHorizontalFlip(), transforms.ToTensor(), transforms.Normalize(mean, std)])

train_dset = ImageFolder("./data/oxford-flowers17/train", transform=train_transform)
train_loader = DataLoader(train_dset, batch_size=batch_size1, num_workers=0, shuffle=True)

val_transform = transforms.Compose([transforms.Scale(224), transforms.CenterCrop(224), transforms.ToTensor(), transforms.Normalize(mean, std)])
val_dset = ImageFolder("./data/oxford-flowers17/val", transform=val_transform)
val_loader = DataLoader(val_dset, batch_size=batch_size1, num_workers=0)

lastmodel = torchvision.models.squeezenet1_1(pretrained=True)
num_classes = len(train_dset.classes)
# (classifier): Sequential(
#     (0): Dropout(p=0.5)
#     (1): Conv2d(512, 1000, kernel_size=(1, 1), stride=(1, 1))
#     (2): ReLU(inplace)
#     (3): AvgPool2d(kernel_size=13, stride=1, padding=0)
#  )
lastmodel.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
lastmodel.num_classes = num_classes
# model.fc = nn.Linear(lastmodel.fc.in_features, num_classes)

# model.type(dtype)
loss_fn = nn.CrossEntropyLoss()
for param in lastmodel.parameters():
    param.requires_grad = False
for param in lastmodel.classifier.parameters():
    param.requires_grad = True


optimizer = torch.optim.Adam(lastmodel.classifier.parameters(), lr=0.001)

Taccuracies = []
Vaccuracies = []
Tlosses = []
for epoch in range(num_epochs):
    # Run an epoch over the training data.
    print('Starting epoch %d' % (epoch + 1))
    loss = run_model(lastmodel, loss_fn, train_loader, optimizer)

    train_acc = calculate_accuracy(lastmodel, train_loader)
    val_acc = calculate_accuracy(lastmodel, val_loader)
    print('Train accuracy: ', train_acc)
    print('Val accuracy: ', val_acc)
    print()
    Taccuracies.append(train_acc)
    Vaccuracies.append(val_acc)
    Tlosses.append(loss)
    

# for param in model.parameters():
#     param.requires_grad = True



# raise NotImplementedError()

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.plot(np.arange(20), [i*100 for i in Taccuracies], label = "Train Accuracy")
plt.xlabel("Iterations")
plt.ylabel("Accuracies (%)")
plt.legend(loc="best")
plt.title("Average Accuracies (%) vs Iterations")
plt.show()

plt.figure()
plt.plot(np.arange(20), [i*100 for i in Vaccuracies], label = "Valid Accuracy")
plt.xlabel("Iterations")
plt.ylabel("Accuracies (%)")
plt.legend(loc="best")
plt.title("Average Accuracies (%) vs Iterations")
plt.show()

plt.figure()
plt.plot(np.arange(20), [i for i in Tlosses], label = "Train Cross Entropy Loss")
plt.xlabel("Iterations")
plt.ylabel("Loss")
plt.legend(loc="best")
plt.title("Average Loss vs Iterations")
plt.show()